In [ ]:
import numpy as np
import random
import plotly.graph_objects as go

from src.projects.Cells.classes.Chemotaxis import Body, Bacteria, EscherichiaColy, Glucose, Lactose, AminoAcid, OxygenBubble

In [ ]:
def generate_cells(cell_class, count, x_range, y_range):
    return [
        cell_class((random.uniform(*x_range), random.uniform(*y_range)))
        for _ in range(count)
    ]

In [ ]:
body = Body([
    *generate_cells(EscherichiaColy, 1, (-50, 50), (-50, 50)),
    *generate_cells(Glucose, 2, (-50, 50), (-50, 50)),
    *generate_cells(Lactose, 1, (-50, 50), (-50, 50)),
    *generate_cells(AminoAcid, 1, (-50, 50), (-50, 50)),
    *generate_cells(OxygenBubble, 4, (-50, 50), (-50, 50)),
])

In [ ]:
path = body.chemotaxis(1000)

In [ ]:
fig = go.Figure()

for cell, positions in path.items():
    xs, ys = zip(*positions)
    name = type(cell).__name__

    if isinstance(cell, Bacteria):
        fig.add_trace(go.Scatter(
            x=xs, y=ys, mode='lines',
            name=f"Bacteria ({name})",
            line=dict(width=3)
        ))
        # Marker Start
        fig.add_trace(go.Scatter(
            x=[xs[0]], y=[ys[0]],
            mode='markers', name="Start",
            marker=dict(color='green', size=10, symbol='circle'),
            showlegend=True  # N'afficher que la première occurrence
        ))
        # Marker End
        fig.add_trace(go.Scatter(
            x=[xs[-1]], y=[ys[-1]],
            mode='markers', name="End",
            marker=dict(color='red', size=10, symbol='x'),
            showlegend=True  # N'afficher que la première occurrence
        ))
    else:
        fig.add_trace(go.Scatter(
            x=xs, y=ys, mode='lines',
            name=f"Target ({name})",
            line=dict(dash='dash')
        ))
        # Marker Start pour les targets
        fig.add_trace(go.Scatter(
            x=[xs[0]], y=[ys[0]],
            mode='markers', name="Start",
            marker=dict(color='blue', size=8, symbol='circle'),
            showlegend=False  # N'afficher pas de nouveau dans la légende
        ))
        # Marker End pour les targets
        fig.add_trace(go.Scatter(
            x=[xs[-1]], y=[ys[-1]],
            mode='markers', name="End",
            marker=dict(color='orange', size=8, symbol='x'),
            showlegend=False  # N'afficher pas de nouveau dans la légende
        ))

fig.update_layout(
    title="Chemotaxis Path of Cells",
    xaxis_title="X Position",
    yaxis_title="Y Position",
    showlegend=True,
    autosize=True,
    margin=dict(l=0, r=0, t=40, b=40),
)

fig.update_yaxes(scaleanchor="x", scaleratio=1)  # Pour garder le rapport XY cohérent
fig.show()

In [ ]:
import plotly.graph_objects as go

# Extraire toutes les cellules et leurs positions au fil du temps
cells = list(path.keys())
steps = len(next(iter(path.values())))  # nombre de frames = durée T

fig = go.Figure()

# Initialisation des traces (pour t=0)
for cell in cells:
    name = type(cell).__name__
    x0, y0 = path[cell][0]

    color = 'green' if isinstance(cell, Bacteria) else 'blue'
    symbol = 'circle' if isinstance(cell, Bacteria) else 'square'

    fig.add_trace(go.Scatter(
        x=[x0], y=[y0],
        mode='markers',
        marker=dict(size=10, color=color, symbol=symbol),
        name=name,
        showlegend=True
    ))

# Créer les frames (une par t)
frames = []
for t in range(steps):
    frame_data = []
    for cell in cells:
        x, y = path[cell][t]
        color = 'green' if isinstance(cell, Bacteria) else 'blue'
        symbol = 'circle' if isinstance(cell, Bacteria) else 'square'

        frame_data.append(go.Scatter(
            x=[x], y=[y],
            mode='markers',
            marker=dict(size=10, color=color, symbol=symbol),
            name=type(cell).__name__,
            showlegend=False  # on montre la légende qu'une seule fois
        ))
    frames.append(go.Frame(data=frame_data, name=str(t)))

# Ajouter les frames à la figure
fig.frames = frames

all_x = [pos[0] for positions in path.values() for pos in positions]
all_y = [pos[1] for positions in path.values() for pos in positions]

xmin, xmax = min(all_x), max(all_x)
ymin, ymax = min(all_y), max(all_y)

# Ajouter une marge
padding = 10
xmin -= padding
xmax += padding
ymin -= padding
ymax += padding


# Slider + bouton play/pause
fig.update_layout(
    title="Chemotaxis Time Evolution",
    xaxis=dict(title="X", range=[xmin, xmax]),
    yaxis=dict(title="Y", scaleanchor="x", scaleratio=1, range=[ymin, ymax]),
    updatemenus=[{
        "type": "buttons",
        "buttons": [
            {"label": "Play", "method": "animate", "args": [None, {"frame": {"duration": 50, "redraw": True}, "fromcurrent": True}]},
            {"label": "Pause", "method": "animate", "args": [[None], {"frame": {"duration": 0}, "mode": "immediate"}]},
        ]
    }],
    sliders=[{
        "steps": [{
            "method": "animate",
            "args": [[str(t)], {"mode": "immediate", "frame": {"duration": 0, "redraw": True}}],
            "label": str(t)
        } for t in range(steps)],
        "transition": {"duration": 0},
        "x": 0, "y": -0.1,
        "currentvalue": {"prefix": "T = "}
    }],
)

fig.show()
